In [5]:
from pyspark.sql.types import StructType,StructField, StringType
import pandas as pd
from pyspark.sql.functions import *
from pyspark.sql.functions import to_timestamp,current_timestamp
from pyspark.sql.types import *



In [2]:
emptyRdd= spark.sparkContext.emptyRDD() #empty rdd

In [28]:
print(type(emptyRdd))

<class 'pyspark.rdd.RDD'>


In [3]:
rdd2=spark.sparkContext.parallelize([])

In [4]:
print(type(rdd2))

<class 'pyspark.rdd.RDD'>


In [14]:
rdd2.count()

0

In [5]:
schema = StructType([
        StructField('Firstname',StringType(),True),
        StructField('middlename',StringType(),True),
        StructField('lastname',StringType(),True),
])

In [6]:
df1= spark.createDataFrame(rdd2,schema)
df1.printSchema()

root
 |-- Firstname: string (nullable = true)
 |-- middlename: string (nullable = true)
 |-- lastname: string (nullable = true)



In [40]:
df2=rdd2.toDF(schema)
df2

DataFrame[Firstname: string, middlename: string, lastname: string]

In [7]:
df2= spark.createDataFrame([],schema)
df2.printSchema()

root
 |-- Firstname: string (nullable = true)
 |-- middlename: string (nullable = true)
 |-- lastname: string (nullable = true)



In [8]:
df2= spark.createDataFrame([],StructType([]))
df2.printSchema()

root



In [81]:
 #1 convert-column-python-list.py
data = [("James","Smith","USA","CA"),("Michael","Rose","USA","NY"), \
    ("Robert","Williams","USA","CA"),("Maria","Jones","USA","FL") \
  ]

In [10]:
columns=["firstname","lastname","country","state"]
df=spark.createDataFrame(data=data,schema=columns)


In [11]:
df.show()
print(df.collect())


24/10/06 03:06:02 INFO src/lib.rs: Boson native library initialized


+---------+--------+-------+-----+
|firstname|lastname|country|state|
+---------+--------+-------+-----+
|    James|   Smith|    USA|   CA|
|  Michael|    Rose|    USA|   NY|
|   Robert|Williams|    USA|   CA|
|    Maria|   Jones|    USA|   FL|
+---------+--------+-------+-----+



[Row(firstname='James', lastname='Smith', country='USA', state='CA'), Row(firstname='Michael', lastname='Rose', country='USA', state='NY'), Row(firstname='Robert', lastname='Williams', country='USA', state='CA'), Row(firstname='Maria', lastname='Jones', country='USA', state='FL')]


In [23]:
states1=df.rdd.map(lambda x: x[3]).collect()
print(states1)

['CA', 'NY', 'CA', 'FL']


In [24]:
from collections import OrderedDict


In [29]:
res=list(OrderedDict.fromkeys(states1)) # convert-column-python-list.py

In [30]:
print(res)

['CA', 'NY', 'FL']


In [38]:
states4= df.select("state").rdd.map(lambda x : x[0]).collect()
states4

['CA', 'NY', 'CA', 'FL']

In [39]:
states4= df.select("state").rdd.flatMap(lambda x : x).collect()
states4

['CA', 'NY', 'CA', 'FL']

In [41]:
df.rdd.map(lambda x: x.lastname).collect()

['Smith', 'Rose', 'Williams', 'Jones']

In [74]:
pandasDF= df.select("firstname","lastname").toPandas()
print(list(pandasDF["firstname"]))
res=list(pandasDF["lastname"])

['James', 'Michael', 'Robert', 'Maria']


In [71]:
import pandas as pd
df10 = pd.DataFrame.from_records()
df10

,0,1,2,3,4,5,6,7
0,S,m,i,t,h,None,None,None
1,R,o,s,e,None,None,None,None
2,W,i,l,l,i,a,m,s
3,J,o,n,e,s,None,None,None


In [79]:
pandasDF= df.select("state").toPandas()["state"]
res=list(pandasDF)
res

['CA', 'NY', 'CA', 'FL']

In [107]:
 #2 singleElement.py
from pyspark.sql.functions import *
from pyspark.sql.functions import to_timestamp,current_timestamp
from pyspark.sql.types import *

schema = StructType([
            StructField("seq", StringType(), True)])
data = [("1",)]
df = spark.createDataFrame(data, schema=schema)
df.show()

+---+
|seq|
+---+
|  1|
+---+



In [126]:
 #3 pandas-pyspark-dataframe.py

import pandas as pd  
data = [['Scott', 50], ['Jeff', 45], ['Thomas', 54],['Ann',34]] 
#data1 = [('Scott', 50), ('Jeff', 45), ('Thomas', 54),('Ann',34)] 

data
pandasDF= pd.DataFrame(data,columns=["name","age"])
pandasDF

,name,age
0,Scott,50
1,Jeff,45
2,Thomas,54
3,Ann,34


In [127]:
mySchema= StructType([
            StructField("first Name",StringType(),True),
            StructField("Age",IntegerType(),True)
        
])

In [128]:
sparkDF2= spark.createDataFrame(pandasDF,mySchema)
sparkDF2.printSchema()
sparkDF2.show()

root
 |-- first Name: string (nullable = true)
 |-- Age: integer (nullable = true)

+----------+---+
|first Name|Age|
+----------+---+
|     Scott| 50|
|      Jeff| 45|
|    Thomas| 54|
|       Ann| 34|
+----------+---+



In [130]:
spark.conf.set("spark.sql.execution.arrow.enabled","true")
spark.conf.set("spark.sql.execution.arrow.pyspark.fallback.enabled","true")


In [140]:
pandasDF2= sparkDF2.select("*").toPandas()
pandasDF2
pandasDF2["first Name"]
list(pandasDF2["Age"])

[50, 45, 54, 34]

In [143]:
test=spark.conf.get("spark.sql.execution.arrow.enabled")
test123=spark.conf.get("spark.sql.execution.arrow.pyspark.fallback.enabled")
test,test123

('true', 'true')

In [161]:
 #4 pyspark-add-month.py
    
data =[("2024-10-11",1) ,("2024-08-09",2),("2024-07-04",3)]
df= spark.createDataFrame(data,schema=["date","increment"]).\
            select("date","increment",\
                   expr("add_months(to_date(date,'yyyy-MM-dd'),cast(increment as Integer))"\
                       ).alias("inc_date")\
                  )
df.show()

+----------+---------+----------+
|      date|increment|  inc_date|
+----------+---------+----------+
|2024-10-11|        1|2024-11-11|
|2024-08-09|        2|2024-10-09|
|2024-07-04|        3|2024-10-04|
+----------+---------+----------+



In [177]:
 #4 pyspark-add-new-column.py
    

data = [('James','Smith','M',3000),
  ('Anna','Rose','F',4100),
  ('Robert','Williams','M',6200), 
]
columns = ["firstname","lastname","gender","salary"]
df = spark.createDataFrame(data=data, schema = columns)
from pyspark.sql.functions import lit
df.withColumn("bonus_percent",lit(0.3)).show()
df.withColumn("bonus_percent",df.salary *0.3).show()
df.withColumn("full_name",concat_ws(',','firstname','lastname')).show()
df.withColumn("current_date",current_date()).show()
df.withColumn("grade",\
              when( 
                    df.salary < 4000,lit('A')\
                  ).when(\
                  (df.salary >= 4000) & (df.salary <= 5000),lit('B')
                        )\
                  .otherwise(lit('C'))).show()


+---------+--------+------+------+-------------+
|firstname|lastname|gender|salary|bonus_percent|
+---------+--------+------+------+-------------+
|    James|   Smith|     M|  3000|          0.3|
|     Anna|    Rose|     F|  4100|          0.3|
|   Robert|Williams|     M|  6200|          0.3|
+---------+--------+------+------+-------------+

+---------+--------+------+------+-------------+
|firstname|lastname|gender|salary|bonus_percent|
+---------+--------+------+------+-------------+
|    James|   Smith|     M|  3000|        900.0|
|     Anna|    Rose|     F|  4100|       1230.0|
|   Robert|Williams|     M|  6200|       1860.0|
+---------+--------+------+------+-------------+

+---------+--------+------+------+---------------+
|firstname|lastname|gender|salary|      full_name|
+---------+--------+------+------+---------------+
|    James|   Smith|     M|  3000|    James,Smith|
|     Anna|    Rose|     F|  4100|      Anna,Rose|
|   Robert|Williams|     M|  6200|Robert,Williams|
+-----

+---------+--------+------+------+------------+
|firstname|lastname|gender|salary|current_date|
+---------+--------+------+------+------------+
|    James|   Smith|     M|  3000|  2024-10-06|
|     Anna|    Rose|     F|  4100|  2024-10-06|
|   Robert|Williams|     M|  6200|  2024-10-06|
+---------+--------+------+------+------------+

+---------+--------+------+------+-----+
|firstname|lastname|gender|salary|grade|
+---------+--------+------+------+-----+
|    James|   Smith|     M|  3000|    A|
|     Anna|    Rose|     F|  4100|    B|
|   Robert|Williams|     M|  6200|    C|
+---------+--------+------+------+-----+



In [194]:
df.select("firstname","salary", lit(0.3).alias("bonus_amount")).show()
df.select("firstname","salary", (df.salary * 0.4).alias("bonus_amount")).show()
df.select("firstname","salary", current_date().alias("todays_date")).show()
df.createOrReplaceTempView("view_df")
spark.sql("select firstname,salary,"+
          "case salary " +
                "when salary < 4000 then 'A' "+
                "when salary >=4000 and salary <=5000 then 'B' " +
                 "else 'C' " +
          "end as grade " +
          "from view_df").show()


+---------+------+------------+
|firstname|salary|bonus_amount|
+---------+------+------------+
|    James|  3000|         0.3|
|     Anna|  4100|         0.3|
|   Robert|  6200|         0.3|
+---------+------+------------+

+---------+------+------------+
|firstname|salary|bonus_amount|
+---------+------+------------+
|    James|  3000|      1200.0|
|     Anna|  4100|      1640.0|
|   Robert|  6200|      2480.0|
+---------+------+------------+

+---------+------+-----------+
|firstname|salary|todays_date|
+---------+------+-----------+
|    James|  3000| 2024-10-06|
|     Anna|  4100| 2024-10-06|
|   Robert|  6200| 2024-10-06|
+---------+------+-----------+

+---------+------+-----+
|firstname|salary|grade|
+---------+------+-----+
|    James|  3000|    C|
|     Anna|  4100|    C|
|   Robert|  6200|    C|
+---------+------+-----+



In [195]:
 #5 pyspark-aggregate.py

simpleData = [("James", "Sales", 3000),
    ("Michael", "Sales", 4600),
    ("Robert", "Sales", 4100),
    ("Maria", "Finance", 3000),
    ("James", "Sales", 3000),
    ("Scott", "Finance", 3300),
    ("Jen", "Finance", 3900),
    ("Jeff", "Marketing", 3000),
    ("Kumar", "Marketing", 2000),
    ("Saif", "Sales", 4100)
  ]
schema = ["employee_name", "department", "salary"]
df = spark.createDataFrame(data=simpleData, schema = schema)
df.printSchema()
df.show(truncate=False)


root
 |-- employee_name: string (nullable = true)
 |-- department: string (nullable = true)
 |-- salary: long (nullable = true)

+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|James        |Sales     |3000  |
|Michael      |Sales     |4600  |
|Robert       |Sales     |4100  |
|Maria        |Finance   |3000  |
|James        |Sales     |3000  |
|Scott        |Finance   |3300  |
|Jen          |Finance   |3900  |
|Jeff         |Marketing |3000  |
|Kumar        |Marketing |2000  |
|Saif         |Sales     |4100  |
+-------------+----------+------+



In [201]:
print(f"approx count distinct:",str(df.select(approx_count_distinct('salary')).collect()[0][0]))

approx count distinct: 6


In [215]:
res= df.agg(collect_list('department'))
df.select(collect_set("department")).show(truncate=False)
res.collect()[0][0]

+---------------------------+
|collect_set(department)    |
+---------------------------+
|[Finance, Sales, Marketing]|
+---------------------------+



['Sales',
 'Sales',
 'Sales',
 'Finance',
 'Sales',
 'Finance',
 'Finance',
 'Marketing',
 'Marketing',
 'Sales']

In [228]:
df2= df.select(countDistinct('department','salary'))
print("Distinct Count of Department &amp; Salary: "+str(df2.collect()[0][0]))

Distinct Count of Department &amp; Salary: 8


In [231]:
df.show()
df.select(first("salary")).show(truncate=False)
df.select(last("salary")).show(truncate=False)
df.select(kurtosis("salary")).show(truncate=False)
df.select(max("salary")).show(truncate=False)
df.select(min("salary")).show(truncate=False)
df.select(mean("salary")).show(truncate=False)
df.select(skewness("salary")).show(truncate=False)
df.select(stddev("salary"), stddev_samp("salary"), \
    stddev_pop("salary")).show(truncate=False)
df.select(sum("salary")).show(truncate=False)
df.select(sumDistinct("salary")).show(truncate=False)
df.select(variance("salary"),var_samp("salary"),var_pop("salary")) \
  .show(truncate=False)

+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|        James|     Sales|  3000|
|      Michael|     Sales|  4600|
|       Robert|     Sales|  4100|
|        Maria|   Finance|  3000|
|        James|     Sales|  3000|
|        Scott|   Finance|  3300|
|          Jen|   Finance|  3900|
|         Jeff| Marketing|  3000|
|        Kumar| Marketing|  2000|
|         Saif|     Sales|  4100|
+-------------+----------+------+

+-------------+
|first(salary)|
+-------------+
|4100         |
+-------------+

+------------+
|last(salary)|
+------------+
|4100        |
+------------+

+-------------------+
|kurtosis(salary)   |
+-------------------+
|-0.6467803030303028|
+-------------------+

+-----------+
|max(salary)|
+-----------+
|4600       |
+-----------+

+-----------+
|min(salary)|
+-----------+
|2000       |
+-----------+

+-----------+
|avg(salary)|
+-----------+
|3400.0     |
+-----------+

+--------------------+
|skewness(salary)  

In [235]:
df10 = spark.createDataFrame([[1],[2],[3]], ["c"])
df10.show()


+---+
|  c|
+---+
|  1|
|  2|
|  3|
+---+



In [236]:
df.select(skewness(df.salary)).show()


+--------------------+
|    skewness(salary)|
+--------------------+
|-0.12041791181069564|
+--------------------+



In [241]:
spark.range(6).select(stddev_pop("id")).show()

+-----------------+
|   stddev_pop(id)|
+-----------------+
|1.707825127659933|
+-----------------+



In [1]:
df = spark.createDataFrame(
    [(680, [[691,1], [692,5]]), (685, [[691,2], [692,2]]), (684, [[691,1], [692,3]])],
    ["product_PK", "products"]
)

In [243]:
df.show()

+----------+--------------------+
|product_PK|            products|
+----------+--------------------+
|       680|[[691, 1], [692, 5]]|
|       685|[[691, 2], [692, 2]]|
|       684|[[691, 1], [692, 3]]|
+----------+--------------------+



In [6]:
get_score = udf(lambda x : x[1],IntegerType())

In [7]:
#other approach 1 using udf
df.withColumn("exploded",explode(df.products)).show()
df_stats = df.withColumn("exploded",explode(df.products))\
                        .withColumn("score",get_score("exploded")).\
                        select(\
                            mean("score").alias("_mean"),\
                            stddev("score").alias("_stddev"))
df_stats.show(1000)

#other approach without udf using in-built array
df.withColumn("val_list",array(df.products.getItem(0).getItem(1),df.products.getItem(1).getItem(1))).show()
df.withColumn("val_list",array(df.products.getItem(0).getItem(1),df.products.getItem(1).getItem(1)))\
            .withColumn("exploded_val",explode("val_list"))\
            .select(mean("exploded_val").alias("_mean"),\
                   stddev("exploded_val").alias("stddev")).show()

24/10/06 21:21:45 INFO src/lib.rs: Boson native library initialized


+----------+--------------------+--------+
|product_PK|            products|exploded|
+----------+--------------------+--------+
|       680|[[691, 1], [692, 5]]|[691, 1]|
|       680|[[691, 1], [692, 5]]|[692, 5]|
|       685|[[691, 2], [692, 2]]|[691, 2]|
|       685|[[691, 2], [692, 2]]|[692, 2]|
|       684|[[691, 1], [692, 3]]|[691, 1]|
|       684|[[691, 1], [692, 3]]|[692, 3]|
+----------+--------------------+--------+



+------------------+-----------------+
|             _mean|          _stddev|
+------------------+-----------------+
|2.3333333333333335|1.505545305418162|
+------------------+-----------------+

+----------+--------------------+--------+
|product_PK|            products|val_list|
+----------+--------------------+--------+
|       680|[[691, 1], [692, 5]]|  [1, 5]|
|       685|[[691, 2], [692, 2]]|  [2, 2]|
|       684|[[691, 1], [692, 3]]|  [1, 3]|
+----------+--------------------+--------+

+------------------+-----------------+
|             _mean|           stddev|
+------------------+-----------------+
|2.3333333333333335|1.505545305418162|
+------------------+-----------------+



In [291]:
spark.range(6).select(stddev_pop("id")).show()

+-----------------+
|   stddev_pop(id)|
+-----------------+
|1.707825127659933|
+-----------------+



In [18]:
 #5 pyspark-array-string.py
columns = ["name","languagesAtSchool","currentState"]
data = [("James,,Smith",["Java","Scala","C++"],"CA"), \
    ("Michael,Rose,",["Spark","Java","C++"],"NJ"), \
    ("Robert,,Williams",["CSharp","VB"],"NV")]

df = spark.createDataFrame(data=data,schema=columns)
df.printSchema()
df.show(truncate=False)

df2= df.withColumn("languagesAtSchool",concat_ws(",","languagesAtSchool"))
df2.printSchema()
df2.show(truncate=False)
df.createOrReplaceTempView("ARRAY_STRING")
spark.sql("select concat_ws(',',languagesAtSchool) as languagesAtSchool  from ARRAY_STRING").show()




root
 |-- name: string (nullable = true)
 |-- languagesAtSchool: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- currentState: string (nullable = true)

+----------------+------------------+------------+
|name            |languagesAtSchool |currentState|
+----------------+------------------+------------+
|James,,Smith    |[Java, Scala, C++]|CA          |
|Michael,Rose,   |[Spark, Java, C++]|NJ          |
|Robert,,Williams|[CSharp, VB]      |NV          |
+----------------+------------------+------------+

root
 |-- name: string (nullable = true)
 |-- languagesAtSchool: string (nullable = false)
 |-- currentState: string (nullable = true)

+----------------+-----------------+------------+
|name            |languagesAtSchool|currentState|
+----------------+-----------------+------------+
|James,,Smith    |Java,Scala,C++   |CA          |
|Michael,Rose,   |Spark,Java,C++   |NJ          |
|Robert,,Williams|CSharp,VB        |NV          |
+----------------+------

In [26]:
 #6 pyspark-arraytype.py
data = [
 ("James,,Smith",["Java","Scala","C++"],["Spark","Java"],"OH","CA"),
 ("Michael,Rose,",["Spark","Java","C++"],["Spark","Java"],"NY","NJ"),
 ("Robert,,Williams",["CSharp","VB"],["Spark","Python"],"UT","NV")
]
schema = StructType([ 
    StructField("name",StringType(),True), 
    StructField("languagesAtSchool",ArrayType(StringType()),True), 
    StructField("languagesAtWork",ArrayType(StringType()),True), 
    StructField("currentState", StringType(), True), 
    StructField("previousState", StringType(), True) 
  ])
df = spark.createDataFrame(data=data,schema=schema)
df.printSchema()
df.show()
#explode
df.select("name", explode("languagesAtSchool")).show()


root
 |-- name: string (nullable = true)
 |-- languagesAtSchool: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- languagesAtWork: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- currentState: string (nullable = true)
 |-- previousState: string (nullable = true)

+----------------+------------------+---------------+------------+-------------+
|            name| languagesAtSchool|languagesAtWork|currentState|previousState|
+----------------+------------------+---------------+------------+-------------+
|    James,,Smith|[Java, Scala, C++]|  [Spark, Java]|          OH|           CA|
|   Michael,Rose,|[Spark, Java, C++]|  [Spark, Java]|          NY|           NJ|
|Robert,,Williams|      [CSharp, VB]|[Spark, Python]|          UT|           NV|
+----------------+------------------+---------------+------------+-------------+

+----------------+------+
|            name|   col|
+----------------+------+
|    James,,Smith|  Java|
|    Ja

In [45]:
#split
df.select(split(df.name,",").alias("nameAsArray")).show()
df2=df.select(explode(split(df.name,",")).alias("nameAsArray"))
df2.show()
#array
df.select(array("currentState","previousState")).show()
#array_contains
df.select(df.name,array_contains(df.languagesAtWork,"Spark").alias("array_contains")).show()

+--------------------+
|         nameAsArray|
+--------------------+
|    [James, , Smith]|
|   [Michael, Rose, ]|
|[Robert, , Williams]|
+--------------------+

+-----------+
|nameAsArray|
+-----------+
|      James|
|           |
|      Smith|
|    Michael|
|       Rose|
|           |
|     Robert|
|           |
|   Williams|
+-----------+

+----------------------------------+
|array(currentState, previousState)|
+----------------------------------+
|                          [OH, CA]|
|                          [NY, NJ]|
|                          [UT, NV]|
+----------------------------------+

+----------------+--------------+
|            name|array_contains|
+----------------+--------------+
|    James,,Smith|          true|
|   Michael,Rose,|          true|
|Robert,,Williams|          true|
+----------------+--------------+



In [86]:
 #7 pyspark-broadcast-dataframe.py
states = {"NY":"New York", "CA":"California", "FL":"Florida"}
broadcastStates = spark.sparkContext.broadcast(states)

data = [("James","Smith","USA","CA"),
    ("Michael","Rose","USA","NY"),
    ("Robert","Williams","USA","CA"),
    ("Maria","Jones","USA","FL"),
        ("Dan","Anshuman","CANADA","BC")
  ]

columns = ["firstname","lastname","country","state"]
df = spark.createDataFrame(data = data, schema = columns)
df.printSchema()
df.show(truncate=False)


def state_convert(code):
    #handle exception for keys not present in dictionary
    if code in states.keys():
        return broadcastStates.value[code]
    

#create df with full state name
dfEnrich = df.rdd.map(lambda x: (x[0],x[1],x[2],state_convert(x[3]))).toDF(columns)
dfEnrich.show()

#res=([x[0] for x in broadcastStates.value.items()])
#print(res)

#filter broadcast variable
filteDf= df.where((df['state'].isin(([x[0] for x in broadcastStates.value.items()]))))
filteDf.show()

root
 |-- firstname: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- country: string (nullable = true)
 |-- state: string (nullable = true)

+---------+--------+-------+-----+
|firstname|lastname|country|state|
+---------+--------+-------+-----+
|James    |Smith   |USA    |CA   |
|Michael  |Rose    |USA    |NY   |
|Robert   |Williams|USA    |CA   |
|Maria    |Jones   |USA    |FL   |
|Dan      |Anshuman|CANADA |BC   |
+---------+--------+-------+-----+

+---------+--------+-------+----------+
|firstname|lastname|country|     state|
+---------+--------+-------+----------+
|    James|   Smith|    USA|California|
|  Michael|    Rose|    USA|  New York|
|   Robert|Williams|    USA|California|
|    Maria|   Jones|    USA|   Florida|
|      Dan|Anshuman| CANADA|      null|
+---------+--------+-------+----------+

+---------+--------+-------+-----+
|firstname|lastname|country|state|
+---------+--------+-------+-----+
|    James|   Smith|    USA|   CA|
|  Michael|    Rose|

In [103]:
 #8 pyspark-cast-column.py
simpleData = [("James",34,"2006-01-01","true","M",3000.60),
    ("Michael",33,"1980-01-10","true","F",3300.80),
    ("Robert",37,"06-01-1992","false","M",5000.50)
  ]

columns = ["firstname","age","jobStartDate","isGraduated","gender","salary"]
df = spark.createDataFrame(data = simpleData, schema = columns)
df.printSchema()
df.show(truncate=False)

df2=df.withColumn("age",col("age").cast(IntegerType()))\
            .withColumn("isGraduated",col("isGraduated").cast(BooleanType()))\
            .withColumn("jobStartDate",col("jobStartDate").cast(DateType()))
df2.printSchema()

df3= df.selectExpr("cast(age as int) as age",
                  "cast(jobStartDate as string) as jobStartDate",
                   "cast(isGraduated as Boolean) as isGraduated",
                  )
df3.printSchema()
df3.createOrReplaceTempView("CastExample")
df4 = spark.sql("SELECT STRING(age),BOOLEAN(isGraduated),DATE(jobStartDate) from CastExample")
df4.printSchema()
df4.show(truncate=False)

root
 |-- firstname: string (nullable = true)
 |-- age: long (nullable = true)
 |-- jobStartDate: string (nullable = true)
 |-- isGraduated: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: double (nullable = true)

+---------+---+------------+-----------+------+------+
|firstname|age|jobStartDate|isGraduated|gender|salary|
+---------+---+------------+-----------+------+------+
|James    |34 |2006-01-01  |true       |M     |3000.6|
|Michael  |33 |1980-01-10  |true       |F     |3300.8|
|Robert   |37 |06-01-1992  |false      |M     |5000.5|
+---------+---+------------+-----------+------+------+

root
 |-- firstname: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- jobStartDate: date (nullable = true)
 |-- isGraduated: boolean (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: double (nullable = true)

root
 |-- age: integer (nullable = true)
 |-- jobStartDate: string (nullable = true)
 |-- isGraduated: boolean (nullable = tru

In [114]:
 #8 pyspark-change-string-double.py
    
simpleData = [("James","34","true","M","3000.6089"),
    ("Michael","33","true","F","3300.8067"),
    ("Robert","37","false","M","5000.5034")
  ]
columns = ["firstname","age","isGraduated","gender","salary"]
df = spark.createDataFrame(data = simpleData, schema = columns)
df.printSchema()
df.show(truncate=False)

df.withColumn("salary",df.salary.cast(DoubleType())).printSchema()
df.withColumn("salary",df.salary.cast("double")).printSchema()
df.selectExpr("cast(salary as double) as salary").printSchema()
res=df.collect()
for row in res:
    print(row["firstname"] +","+str(row["age"]))

root
 |-- firstname: string (nullable = true)
 |-- age: string (nullable = true)
 |-- isGraduated: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: string (nullable = true)

+---------+---+-----------+------+---------+
|firstname|age|isGraduated|gender|salary   |
+---------+---+-----------+------+---------+
|James    |34 |true       |M     |3000.6089|
|Michael  |33 |true       |F     |3300.8067|
|Robert   |37 |false      |M     |5000.5034|
+---------+---+-----------+------+---------+

root
 |-- firstname: string (nullable = true)
 |-- age: string (nullable = true)
 |-- isGraduated: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: double (nullable = true)

root
 |-- firstname: string (nullable = true)
 |-- age: string (nullable = true)
 |-- isGraduated: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: double (nullable = true)

root
 |-- salary: double (nullable = true)

James,34
Michael,33
Robert,37


In [136]:
 #9 pyspark-string-date.py

df=spark.createDataFrame([["02-03-2013"],["05-06-2023"]],["input"])
df.show()
df.printSchema()
df=df.select("input",to_date("input","dd-MM-yyyy").alias("date"))
df.printSchema()
df.show()

+----------+
|     input|
+----------+
|02-03-2013|
|05-06-2023|
+----------+

root
 |-- input: string (nullable = true)

root
 |-- input: string (nullable = true)
 |-- date: date (nullable = true)

+----------+----------+
|     input|      date|
+----------+----------+
|02-03-2013|2013-03-02|
|05-06-2023|2023-06-05|
+----------+----------+



In [152]:
 #10 pyspark-string-timestamp.py
df=spark.createDataFrame(
        data = [ ("1","2019-06-24 12:01:19.333")],
        schema=["id","input_timestamp"])
df.printSchema()
df= df.withColumn("timestamp",to_timestamp("input_timestamp"))
df.printSchema()
df.show(truncate=False)
df= df.withColumn("timestamp_as_date",to_timestamp("input_timestamp").cast(DateType()))
df= df.withColumn("timestamp_as_string",to_timestamp("input_timestamp").cast(StringType()))
df= df.withColumn("timestamp_as_string",to_timestamp("input_timestamp").cast("string"))
df.printSchema()
df.show(truncate=False)
df.select(to_timestamp(lit('2024-11-09 12:01:19.000'),'yyyy-MM-dd HH:mm:ss.SSSS').alias("timestamp_new")).show()
spark.sql("select to_timestamp('2019-06-24 12:01:19.333','yyyy-MM-dd HH:mm:ss.SSSS') as timestamp_new1").show(truncate=False)
spark.sql("select to_timestamp('2019-06-24 12:01:19.333') as timestamp_new2").show(truncate=False)

root
 |-- id: string (nullable = true)
 |-- input_timestamp: string (nullable = true)

root
 |-- id: string (nullable = true)
 |-- input_timestamp: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)

+---+-----------------------+-----------------------+
|id |input_timestamp        |timestamp              |
+---+-----------------------+-----------------------+
|1  |2019-06-24 12:01:19.333|2019-06-24 12:01:19.333|
+---+-----------------------+-----------------------+

root
 |-- id: string (nullable = true)
 |-- input_timestamp: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestamp_as_date: date (nullable = true)
 |-- timestamp_as_string: string (nullable = true)

+---+-----------------------+-----------------------+-----------------+-----------------------+
|id |input_timestamp        |timestamp              |timestamp_as_date|timestamp_as_string    |
+---+-----------------------+-----------------------+-----------------+-------------------

In [158]:
#11 pyspark-string-to-array.py
data = [("James, A, Smith","2018","M",3000),
            ("Michael, Rose, Jones","2010","M",4000),
            ("Robert,K,Williams","2010","M",4000),
            ("Maria,Anne,Jones","2005","F",4000),
            ("Jen,Mary,Brown","2010","",-1)
            ]

columns=["name","dob_year","gender","salary"]
df=spark.createDataFrame(data,columns)
df.printSchema()
df.show(truncate=False)

root
 |-- name: string (nullable = true)
 |-- dob_year: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: long (nullable = true)

+--------------------+--------+------+------+
|name                |dob_year|gender|salary|
+--------------------+--------+------+------+
|James, A, Smith     |2018    |M     |3000  |
|Michael, Rose, Jones|2010    |M     |4000  |
|Robert,K,Williams   |2010    |M     |4000  |
|Maria,Anne,Jones    |2005    |F     |4000  |
|Jen,Mary,Brown      |2010    |      |-1    |
+--------------------+--------+------+------+



In [168]:
#12 pyspark-struct-to-map.py

data = [ ("36636","Finance",(3000,"USA")), 
    ("40288","Finance",(5000,"IND")), 
    ("42114","Sales",(3900,"USA")), 
    ("39192","Marketing",(2500,"CAN")), 
    ("34534","Sales",(6500,"USA")) ]

schema = StructType([
            StructField('id',StringType(),True),
            StructField('dept',StringType(),True),
            StructField('properties', StructType([
                            StructField('salary',IntegerType(),True),
                            StructField('location',StringType(),True)
                            ]))  
])
df= spark.createDataFrame(data=data,schema=schema)
df.printSchema()
df.show()
#Convert struct type to Map
df= df.withColumn("propertiesMap",create_map(
                    lit("salary"),col("properties.salary"),
                    lit("country"),col("properties.location")
                    ))
df.show()
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- dept: string (nullable = true)
 |-- properties: struct (nullable = true)
 |    |-- salary: integer (nullable = true)
 |    |-- location: string (nullable = true)

+-----+---------+-----------+
|   id|     dept| properties|
+-----+---------+-----------+
|36636|  Finance|{3000, USA}|
|40288|  Finance|{5000, IND}|
|42114|    Sales|{3900, USA}|
|39192|Marketing|{2500, CAN}|
|34534|    Sales|{6500, USA}|
+-----+---------+-----------+

+-----+---------+-----------+--------------------+
|   id|     dept| properties|       propertiesMap|
+-----+---------+-----------+--------------------+
|36636|  Finance|{3000, USA}|{salary -> 3000, ...|
|40288|  Finance|{5000, IND}|{salary -> 5000, ...|
|42114|    Sales|{3900, USA}|{salary -> 3900, ...|
|39192|Marketing|{2500, CAN}|{salary -> 2500, ...|
|34534|    Sales|{6500, USA}|{salary -> 6500, ...|
+-----+---------+-----------+--------------------+

root
 |-- id: string (nullable = true)
 |-- dept: string (null

In [187]:

data = [ ("36636","Finance",(3000,"USA")), 
    ("40288","Finance",(5000,"IND")), 
    ("42114","Sales",(3900,"USA")), 
    ("39192","Marketing",(2500,"CAN")), 
    ("34534","Sales",(6500,"USA")) ]

schema = StructType([
        StructField("id",StringType(),True),
        StructField("dept",StringType(),True),
        StructField("properties", StructType([
                StructField("salary",IntegerType(),True),
                StructField("country",StringType(),True)
                ]))
])

df= spark.createDataFrame(data=data,schema=schema)
df.printSchema()
df.select(col('properties.salary'),col('properties.country')).show()
df=df.withColumn("propertiesMap", create_map(
                    lit("salary"),col('properties.salary'),
                    lit("country"),col('properties.country'),
                    ))
df.show(truncate=False)

root
 |-- id: string (nullable = true)
 |-- dept: string (nullable = true)
 |-- properties: struct (nullable = true)
 |    |-- salary: integer (nullable = true)
 |    |-- country: string (nullable = true)

+------+-------+
|salary|country|
+------+-------+
|  3000|    USA|
|  5000|    IND|
|  3900|    USA|
|  2500|    CAN|
|  6500|    USA|
+------+-------+

+-----+---------+-----------+--------------------------------+
|id   |dept     |properties |propertiesMap                   |
+-----+---------+-----------+--------------------------------+
|36636|Finance  |{3000, USA}|{salary -> 3000, country -> USA}|
|40288|Finance  |{5000, IND}|{salary -> 5000, country -> IND}|
|42114|Sales    |{3900, USA}|{salary -> 3900, country -> USA}|
|39192|Marketing|{2500, CAN}|{salary -> 2500, country -> CAN}|
|34534|Sales    |{6500, USA}|{salary -> 6500, country -> USA}|
+-----+---------+-----------+--------------------------------+

